In [ ]:
# Importing packages

import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Sklearn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Models
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from lightgbm.callback import early_stopping

# Optuna for hyperparameter optimization
import optuna

**Objective**

This notebook is designed to forecast store sales 16 days into the future. The data holds sales data per day collected between 2013 and July 2017. It is constructed as follows: each row gives the sales and number of items on promotion,on a specific day for a specific store and in a specific product family. Other data sources provide information about national, regional or local holidays, location and properties of the stores, oil prices, and total transactions per day per store.

**Approach**

The task involves forecasting different time-series, that is, for each pair (store number, product family) we have a distinct time series. However, we will use machine-learning models to learn the dynamics of these time series together, since we believe that there are similarities between the dynamics. In order to enable the models to learn patterns that apply across different pairs of (store number, product family), we choose to feed our model with the entire data, using store number and family as features.

We will train different models to predict the sales for each of the 16 days in the test file. We will use the last date of the train file as the prediction origin.

**Data ingestion and preparation**

In the following, we read the different data sources and prepare them. The following preliminary processing is handled:
1. converting the 'date' field to datetime format
2. default values are included for missing dates (no sales on Christmas)
3. missing oil prices are filled through linear interpolation.
4. missing transactions entries are filled by sales / average sales per transaction per store.
5. holidays and events are binned (for example, all football events are binned together), and are loaded to the data frame for stores for which they are geographically relevant.

All these processed data sources are merged into a full data frame that contains the underlying information, from which other features are extracted.

In [ ]:
# Reading in provided source data

train_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv",index_col='id')
holidays_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
oil_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
stores_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
transactions_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
test_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv",index_col='id')

In [ ]:
# Correct date format

train_df['date'] = pd.to_datetime(train_df['date'])
holidays_df['date'] = pd.to_datetime(holidays_df['date'])
oil_df['date'] = pd.to_datetime(oil_df['date'])
transactions_df['date'] = pd.to_datetime(transactions_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

In [ ]:
# Find missing dates in train_df (Christmas). Add entries for these dates with sales of 0 for each store_nbr, family.

missing_dates = pd.date_range(start=train_df['date'].min(), end=train_df['date'].max()).difference(train_df['date'])
temporary_df = train_df[(train_df['date']>='2017-02-03') & (train_df['date']<=pd.to_datetime('2017-02-03')+DateOffset(days=len(missing_dates)-1))].copy()
temporary_df[['sales','onpromotion']] = 0
for i in range(len(missing_dates)):
    temporary_df.loc[temporary_df['date']==pd.to_datetime('2017-02-03')+DateOffset(days=i),'date'] = missing_dates[i]
train_df =  pd.concat([train_df, temporary_df], axis=0)

In [ ]:
# Concatenate train_df and test_df. Sort by date.
full_df = pd.concat([train_df, test_df], axis = 0)
full_df = full_df.sort_values(by=['date','store_nbr','family'],ignore_index=True)

In [ ]:
# Interpolate oil prices linearly where missing.
calendar_df = pd.DataFrame(pd.date_range(start='2013-01-01', end='2017-08-15'), columns = ['date'])
oil_df = pd.merge(left=calendar_df,right=oil_df,on='date',how='left')
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].interpolate()

In [ ]:
# Preparing transaction data

# Transaction data is missing for some dates and stores. Use sales data and average sales per transaction to estimate transaction count.

# Calculate sum of sales per store
transactions_df = pd.merge(left=transactions_df,right=pd.DataFrame(train_df.groupby(['date','store_nbr'])['sales'].sum()).reset_index(),on=['date', 'store_nbr'],how='outer')

# Calculate sales per transaction (per day and store)
transactions_df['sales_per_transaction'] = transactions_df['sales'] / transactions_df['transactions']

# Calculate average sales per transaction per store
store_ratio = transactions_df.groupby('store_nbr')['sales_per_transaction'].mean().to_dict()

# Fill missing transactions with sales / average sales per transaction per store.
transactions_df['transactions'] = transactions_df['transactions'].fillna(transactions_df['sales']/transactions_df['store_nbr'].map(store_ratio))
transactions_df = transactions_df[['date', 'store_nbr', 'transactions']]

In [ ]:
# Preparing holiday data

new_holidays_df = holidays_df[holidays_df['transferred']==False]
new_holidays_df = new_holidays_df.groupby(['date','locale','locale_name']).sum()
new_holidays_df = new_holidays_df.reset_index()

# Remove transferred holidays as they resemble workdays
new_holidays_df = holidays_df[holidays_df['transferred']==False]

# Binning football holidays (before anticipated holidays to remove '-')
new_holidays_df.loc[(new_holidays_df['description'].str.contains('futbol')), 'description'] = 'Futbol'

# Removing anticipated holidays because we will use future features
new_holidays_df = new_holidays_df.loc[new_holidays_df['description'].str.find('-')==-1]

# Binning local holidays
new_holidays_df.loc[(new_holidays_df['locale']=='Local') & (new_holidays_df['description'].str.contains('Fundacion')), 'description'] = 'Fundacion'
new_holidays_df.loc[(new_holidays_df['locale']=='Local') & (new_holidays_df['description'].str.contains('Cantonizacion')), 'description'] = 'Cantonizacion'
new_holidays_df.loc[(new_holidays_df['locale']=='Local') & (new_holidays_df['description'].str.contains('Independencia')), 'description'] = 'Independencia'

# Binning regional holidays
new_holidays_df.loc[(new_holidays_df['locale']=='Regional') & (new_holidays_df['description'].str.contains('Provincializacion')), 'description'] = 'Provincializacion'

# Binning national holidays
new_holidays_df.loc[(new_holidays_df['locale']=='National') & (new_holidays_df['description'].str.contains('Terremoto')), 'description'] = 'Earthquake'
new_holidays_df.loc[(new_holidays_df['locale']=='National') & (new_holidays_df['description'].str.contains('Recupero')), 'description'] = 'Work Day'
new_holidays_df['description'] = new_holidays_df['description'].str.replace('Traslado ','')

# Removing past holidays as we use lagged features
new_holidays_df = new_holidays_df.loc[new_holidays_df['description'].str.find('+')==-1]

# Joining descriptions if more than one holiday applies in same location
check_df = new_holidays_df.groupby(['date','locale','locale_name'])['description'].apply(', '.join).reset_index()
new_holidays_df = new_holidays_df.drop('description', axis=1)
new_holidays_df = new_holidays_df.merge(check_df, on=['date','locale','locale_name'], how='left')
new_holidays_df = new_holidays_df.drop_duplicates(subset=['date','locale','locale_name'])

In [ ]:
# Preparing store data

# Renaming type in stores_df to avoid any confusion with holiday types.
stores_df = stores_df.rename(columns={'type':'store_type'})

In [ ]:
# Merging dataframes

full_df = pd.merge(left=full_df,right=stores_df,on='store_nbr',how='left')
full_df = pd.merge(left=full_df,right=oil_df,on='date',how='left')
full_df = pd.merge(left=full_df,right=transactions_df,on=['date','store_nbr'],how='left')
full_df = pd.merge(left=full_df,right=new_holidays_df.loc[new_holidays_df['locale']=='National',['date','description','type']],
                                    on='date',how='left')
full_df = full_df.rename(columns={'description':'national_holiday_description','type':'national_holiday_type'})
full_df = pd.merge(left=full_df,right=new_holidays_df.loc[new_holidays_df['locale']=='Regional',['date','locale_name','description','type']],
                                    left_on=['date','state'],right_on=['date','locale_name'],how='left')
full_df = full_df.rename(columns={'description':'regional_holiday_description','type':'regional_holiday_type'})
full_df = full_df.drop(axis='columns',labels='locale_name')
full_df = pd.merge(left=full_df,right=new_holidays_df.loc[new_holidays_df['locale']=='Local',['date','locale_name','description','type']],
                                    left_on=['date','city'],right_on=['date','locale_name'],how='left')
full_df = full_df.rename(columns={'description':'local_holiday_description','type':'local_holiday_type'})
full_df = full_df.drop(axis='columns',labels='locale_name')

In [ ]:
# Place default string value if there is no holiday.
holiday_features = ['national_holiday_description','national_holiday_type',
                'regional_holiday_description','regional_holiday_type',
                'local_holiday_description','local_holiday_type']
full_df[holiday_features] = full_df[holiday_features].fillna('No holiday')

After processing, we verify that each possible triplet (date, store number, family) appears exactly once in the data frame.

In [ ]:
# Sanity check: After processing, each pair of store_nbr, family has exactly one entry for each date.

# Each possible triple appears, because the number of possible distinct triples == number of appearing distinct triples.

x = len(set(list(zip(full_df['date'], full_df['store_nbr'], full_df['family']))))

print(full_df['date'].nunique()*full_df['store_nbr'].nunique()*full_df['family'].nunique() == x)

# Each triple appears exaxtly once.

print(x == len(list(zip(full_df['date'], full_df['store_nbr'], full_df['family']))))

**Feature engineering**

We will use the last date of the train file (2017-08-15) as the prediction origin.
So all features for our models need to be available in the rows for this date.

We create the following features:
1. day of the month
2. month of year, sine and cosine with period one year to capture seasonality
3. year
4. day of week, sine and cosine with period one week to capture seasonality
5. day of payment cycle, sine and cosine with period two weeks to capture seasonality
6. past sales shifted (autoregressive dynamics)
7. moving average of sales with window sizes of one week, one month

We also create the following future features (future data that is known at the forecast origin).
There is some danger of leakage, but we assume promotions are known two weeks in advance:

8. holidays in the prediction period
9. promotions in the prediction period
10. moving average of onpromotion over the prediction period

In [ ]:
prediction_date = train_df['date'].max()

In [ ]:
# Time features

full_df['day'] = full_df['date'].dt.day
full_df['month'] = full_df['date'].dt.month
full_df['year'] = full_df['date'].dt.year

full_df['day_of_week'] = full_df['date'].dt.day_of_week
full_df['sin_day_of_week'] = np.sin(2*np.pi*(full_df['day_of_week']/7))
full_df['cos_day_of_week'] = np.cos(2*np.pi*(full_df['day_of_week']/7))

full_df['sin_month_of_year'] = np.sin(2*np.pi*(full_df['date'].dt.month/12))
full_df['cos_month_of_year'] = np.cos(2*np.pi*(full_df['date'].dt.month/12))

full_df['day_of_pay_cycle'] = (full_df['day'] - 15)*(full_df['day'] >= 15)+(full_df['day']+1)*(full_df['day'] < 15)
full_df['sin_day_of_pay_cycle'] = np.sin(2*np.pi*(full_df['day_of_pay_cycle']/15))
full_df['cos_day_of_pay_cycle'] = np.cos(2*np.pi*(full_df['day_of_pay_cycle']/15))

# Lag features

# Sales from every day of the last week

shifted_sales = {
    f'sales_lag_{i}': full_df.groupby(['store_nbr', 'family'])['sales'].shift(i) 
    for i in range(1, 8)
}

full_df = pd.concat([full_df, pd.DataFrame(shifted_sales)], axis=1)

# Sales on the same day of the month, from last month

full_df['next_month'] = full_df['date']+ DateOffset(months=1)
full_df  = pd.merge(left=full_df,right=full_df[['next_month','store_nbr','family', 'sales']], 
                               left_on=['date','store_nbr', 'family'], 
                               right_on=['next_month','store_nbr','family'], 
                               how='left')
full_df = full_df.rename(columns={'sales_x':'sales','sales_y':'sales_last_month'})
full_df = full_df.drop(labels=['next_month_x','next_month_x'], axis = 1)

# Rolling averages of sales data

full_df['MA_last_week'] = full_df.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=7).mean())
full_df['MA_last_month'] = full_df.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=30).mean())

In [ ]:
# Creating future features.

holiday_features = ['national_holiday_description','national_holiday_type',
                    'regional_holiday_description','regional_holiday_type',
                    'local_holiday_description','local_holiday_type']
future_features = holiday_features + ['onpromotion']

full_df = full_df.drop(columns=[f'{feature}_step_{i}' for feature in future_features for i in range(0, 17)], errors='ignore')

shifted_data = {
    f'{feature}_step_{i}': full_df.groupby(['store_nbr', 'family'])[f'{feature}'].shift(-i) 
    for feature in future_features for i in range(0, 17)
}

full_df = pd.concat([full_df, pd.DataFrame(shifted_data)], axis=1)

# Rolling average of future promotions
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=16)
full_df['MA_future_promotions'] = full_df.groupby(['store_nbr', 'family'])['onpromotion'].transform(lambda x: x.rolling(window=indexer).mean())

In [ ]:
# Future features are also available with column name suffix '_step_0'.

full_df = full_df.drop(columns=future_features)

In [ ]:
# Here we make the distinction between numerial and categorical features.

# Categorical features are 

future_categorical_features = [f'{feature}_step_{i}' for feature in holiday_features for i in range(0, 17)]
categorical_features = ['store_nbr', 'family', 'day_of_week'] # 'city', 'state', 'store_type', 'cluster'

full_df[categorical_features] = full_df[categorical_features].astype('category')
full_df[future_categorical_features] = full_df[future_categorical_features].astype('category')


sales_features = [f'sales_lag_{i}' for i in range(1, 8)]
numerical_features = ['dcoilwtico', 'transactions',  
                      'day', 'month', 'year', 
                      'sin_day_of_week', 'cos_day_of_week', 'sin_month_of_year', 'cos_month_of_year', 
                      'day_of_pay_cycle', 'sin_day_of_pay_cycle', 'cos_day_of_pay_cycle', 
                      'sales_last_month', 'MA_last_week', 'MA_last_month',
                      'MA_future_promotions'] + sales_features

feature_list = []

# For each day we will train a different models.
# To only use the most relevant features for each model, We create a list, that stores in position 

for i in range(1, 17):
    feature_list.append(numerical_features + categorical_features 
                        + [f'{feature}_step_{j}' for feature in holiday_features for j in range(i-1, min(i+5,17))]
                        + [f'onpromotion_step_{i}'])

In [ ]:
# Examples

str_nbr = 7
fml = 'BEVERAGES'

sns.scatterplot(data = full_df[(full_df['store_nbr']== str_nbr)& (full_df['family']== fml)], x='sales_lag_1', y='sales')
# sns.scatterplot(data = full_df[(full_df['store_nbr']== str_nbr)& (full_df['family']== fml)], x='sales_lag_7', y='sales')
# sns.scatterplot(data = full_df[(full_df['store_nbr']== str_nbr)& (full_df['family']== fml)], x='dcoilwtico', y='sales')

The figure above motivates the use of the shifted sales. There is clearly a strong positive correlation between yesterday's sales and today's. This suggests that there might be an autoregressice component to the dynamics. This feature, and others, are further justified below when the F-score is computed.

**Target**

The objective is to forecast 16 days into the future. To this end, we will train 16 different models, one for each horizon. This is done by building a full dataframe which is composed of the features and target for all models. The features were discussed above, and the targets for each date are the sales in the 16 days that follow (negtively shifted sales).

In [ ]:
# Create target data

full_df = full_df.drop(columns=[f'sales_step_{i}' for i in range(1, 17)], errors='ignore')

shifted_data = {
    f'sales_step_{i}': full_df.groupby(['store_nbr', 'family'])['sales'].shift(-i) 
    for i in range(1, 17)
}

full_df = pd.concat([full_df, pd.DataFrame(shifted_data)], axis=1)

# Take log on target

full_df['log_sales'] = np.log(1+full_df['sales'])
for i in range(1,17):
    full_df[f'log_sales_step_{i}'] = np.log(1+full_df[f'sales_step_{i}'])

In [ ]:
# Filter set to remove rows with missing features or targets
subset_na = [f'log_sales_step_{i}' for i in range(1,17)]+numerical_features
X = full_df.dropna(axis=0,subset=subset_na)

In [ ]:
# Restrict data

# X = X[X['year']>2014]

In [ ]:
# Target
target_list = [f'log_sales_step_{i}' for i in range(1,17)]
y = X[target_list]

In [ ]:
# Train test split

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

In [ ]:
# Idea:
# 1. Train different types of models
# 2. Hyperparameter optimization
# 3. After having chosen good hyperparameters, train each chosen model on a separate split.
# 4. Combine (probably just average) models.

In [ ]:
# Hyperparameter optimization for xgb
# Ideally, we would find the optimal hyperparameters for each model. This takes too much time.  Choose 7.

# def objective_xgb(trial):
#     params={
#         'max_depth':trial.suggest_int('max_depth',6,15),
#         'min_child_weight':trial.suggest_float('min_child_weight',1e-2,100,log=True),
#         'subsample':trial.suggest_float('subsample',0.5,1),
#         'learning_rate':trial.suggest_float('learning_rate',0.01,0.3,log=True),
#         'colsample_bytree':trial.suggest_float('colsample_bytree',0.5,1)
#     }
#     model = XGBRegressor(
#         **params,
#         n_estimators=1000,
#         early_stopping_rounds=5,
#         eval_metric='rmse',
#         tree_method='hist',
#         enable_categorical = True
#     )
#     model.fit(
#         X_train[feature_list[7]], 
#         y_train[f'log_sales_step_8'], 
#         eval_set=[(X_valid[feature_list[7]], y_valid[f'log_sales_step_8'])], 
#         verbose = 0)
#     preds = model.predict(X_valid[feature_list[7]])
#     return mean_squared_error(y_valid[f'log_sales_step_8'],preds)

# study_xgb = optuna.create_study(direction='minimize')
# study_xgb.optimize(objective_xgb,n_trials=50)
# best_params_xgb = study_xgb.best_params
# print(best_params_xgb)

In [ ]:
# Result of hyperparameter optimization for xgb

# With 50 iterations:

# best_params_xgb={
#      'max_depth': 11, 
#      'min_child_weight': 5.716785274742769, 
#      'subsample': 0.9234688380913888, 
#      'learning_rate': 0.040487705693284505, 
#      'colsample_bytree': 0.6294748159852215
#     }

# With 35 iterations:

# best_params_xgb={
#         'max_depth':14,
#         'min_child_weight':4.990640605482132,
#         'subsample':0.8814379905410369,
#         'learning_rate':0.03259546572,
#         'colsample_bytree':0.8437365655068993
#     }

# Heuristic defaults:

best_params_xgb={
    'max_depth': 8,
    'min_child_weight': 5,
    'subsample': 0.8,
    'learning_rate': 0.05,
    'colsample_bytree': 0.8
    }

In [ ]:
# Hyperparameter optimization for lgbm
# Ideally, we would find the optimal hyperparameters for each model. This takes too much time.  Choose 7.

# from lightgbm import LGBMRegressor
# from lightgbm.callback import early_stopping

# def objective_lgbm(trial):
#     params={
#         'num_leaves':trial.suggest_int('num_leaves',30,150),
#         'min_child_weight':trial.suggest_float('min_child_weight',1e-3,100,log=True),
#         'subsample':trial.suggest_float('subsample',0.5,1),
#         'learning_rate':trial.suggest_float('learning_rate',0.01,0.3,log=True),
#         'colsample_bytree':trial.suggest_float('colsample_bytree',0.5,1)
#     }
#     model = LGBMRegressor(
#         **params,
#         n_estimators=1000
#     )
#     model.fit(
#         X_train[feature_list[7]],
#         y_train[f'log_sales_step_8'],
#         eval_set=[(X_valid[feature_list[7]], y_valid[f'log_sales_step_8'])],
#         callbacks=[early_stopping(stopping_rounds=5)],
#         eval_metric='rmse'
#     )
#     preds = model.predict(X_valid[feature_list[7]])
#     return mean_squared_error(y_valid[f'log_sales_step_8'],preds)

# study_lgbm = optuna.create_study(direction='minimize')
# study_lgbm.optimize(objective_lgbm,n_trials=50)
# best_params_lgbm = study_lgbm.best_params
# print(best_params_lgbm)

In [ ]:
# Result of hyperparameter optimization for lgbm

# With 50 iterations: 

# best_params_lgbm={
#     'num_leaves': 141, 
#      'min_child_weight': 0.0013941886951568947, 
#      'subsample': 0.892295408305398, 
#      'learning_rate': 0.10205166138848568, 
#      'colsample_bytree': 0.6582560601895524
#     }

# With 35 iterations:

# best_params_lgbm={
#         'num_leaves':141,
#         'min_child_weight':0.02362221828066401,
#         'subsample':0.6927354612523271,
#         'learning_rate':0.10616785590747908,
#         'colsample_bytree':0.7840718293546729
#     }

# Heuristic defaults:

best_params_lgbm={
        'num_leaves':31,
        'min_child_weight':5,
        'subsample':0.8,
        'learning_rate':0.05,
        'colsample_bytree':0.8
    }

In [ ]:
# Hyperparameter optimization for cat
# Ideally, we would find the optimal hyperparameters for each model. This takes too much time. Choose 7.

# #pip install optuna-integration[catboost]
# #from optuna.integration import CatBoostPruningCallback

#def objective_cat(trial):
#    params={
#        'depth':trial.suggest_int('depth',4,10),
#        'random_strength':trial.suggest_float('random_strength',1e-9,10,log=True),
#        'bagging_temperature':trial.suggest_float('bagging_temperature',0,10),
#        'learning_rate':trial.suggest_float('learning_rate',0.01,0.3,log=True),
#        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10.0, log=True)
#    }
#    model = CatBoostRegressor(
#        **params,
#        loss_function='RMSE',
#        task_type="GPU",        # <-- enable GPU
#        devices='0,1',
#        iterations=500
#    )
#    # pruning_callback = CatBoostPruningCallback(trial, 'RMSE')
#    model.fit(
#        X_train[feature_list[7]],
#        y_train[f'log_sales_step_8'],
#        cat_features = list(set(categorical_features+future_categorical_features) & set(feature_list[7])),
#        eval_set=(X_valid[feature_list[7]], y_valid[f'log_sales_step_8']),
#        early_stopping_rounds=5,
#        # callbacks=[pruning_callback],
#        verbose = 50)
#    preds = model.predict(X_valid[feature_list[7]])
#    return mean_squared_error(y_valid[f'log_sales_step_8'],preds)

# #study_cat = optuna.create_study(direction='minimize',pruner=optuna.pruners.MedianPruner(n_warmup_steps=50))
#study_cat = optuna.create_study(direction='minimize')
#study_cat.optimize(objective_cat,n_trials=50)
#best_params_cat = study_cat.best_params
#print(best_params_cat)

In [ ]:
# Result of hyperparameter optimization for catboost

# With 50 iterations:

# best_params_cat={
#        'depth':9,
#        'random_strength':0.05626974752238471,
#        'bagging_temperature':0.09747368919292904,
#        'learning_rate':0.236652999497516,
#        'l2_leaf_reg':4.580022980524382}

# Heuristic defaults:

best_params_cat={
       'depth':8,
       'random_strength':1,
       'bagging_temperature':1,
       'learning_rate':0.05,
       'l2_leaf_reg':3}

**Training**

Here we train 16 models for 16 different horizons. We use three booting models: XG boost, light gradient boosting model, and cat boost - and we average these with weights that are related with the validation score of the models.

In [ ]:
# Training xgboost. We do one split per type of model, to save time.

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

models_xgb = {}
weights_xgb = []
for i in range(1,17):
    print(i)
    model = XGBRegressor(
        **best_params_xgb,
        n_estimators=1000,
        tree_method='hist',     
        device = 'cuda',
        early_stopping_rounds=5,
        eval_metric='rmse',
        enable_categorical = True
    )
    model.fit(
        X_train[feature_list[i-1]], 
        y_train[f'log_sales_step_{i}'], 
        eval_set=[(X_valid[feature_list[i-1]], y_valid[f'log_sales_step_{i}'])], 
        verbose = 0)
    preds = model.predict(X_valid[feature_list[i-1]])
    weights_xgb.append(1/mean_squared_error(y_valid[f'log_sales_step_{i}'],preds))
    models_xgb[i] = model

In [ ]:
# Train lightgbm. We do one split per type of model, to save time.

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

models_lgbm = {}
weights_lgbm = []
for i in range(1, 17):
    print(i)
    model = LGBMRegressor(
        **best_params_lgbm,    
        device='gpu',           
        gpu_platform_id=0,      
        gpu_device_id=0, 
        n_estimators=1000
    )
    model.fit(
        X_train[feature_list[i - 1]],
        y_train[f'log_sales_step_{i}'],
        eval_set=[(X_valid[feature_list[i - 1]], y_valid[f'log_sales_step_{i}'])],
        callbacks=[early_stopping(stopping_rounds=5)],
        eval_metric='rmse'
    )
    preds = model.predict(X_valid[feature_list[i-1]])
    weights_lgbm.append(1/mean_squared_error(y_valid[f'log_sales_step_{i}'],preds))
    models_lgbm[i] = model

In [ ]:
# Train catboost. We do one split per type of model, to save time.

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

models_cat = {}
weights_cat = []
for i in range(1,17):
    print(i)
    model = CatBoostRegressor(
        **best_params_cat,
        loss_function='RMSE',
        task_type="GPU",        # <-- enable GPU
        devices='0',            # use first GPU
        iterations=1000
    )
    model.fit(
        X_train[feature_list[i-1]],
        y_train[f'log_sales_step_{i}'],
        cat_features = list(set(categorical_features+future_categorical_features) & set(feature_list[i-1])),
        eval_set=(X_valid[feature_list[i-1]], y_valid[f'log_sales_step_{i}']),
        early_stopping_rounds = 5,
        verbose = 50)
    preds = model.predict(X_valid[feature_list[i-1]])
    weights_cat.append(1/mean_squared_error(y_valid[f'log_sales_step_{i}'],preds))
    models_cat[i] = model

In [ ]:
print(weights_xgb)
print(weights_lgbm)
print(weights_cat)

In [ ]:
# computing the weights for the weighted average

weights_xgb = np.array(weights_xgb)
weights_lgbm = np.array(weights_lgbm)
weights_cat = np.array(weights_cat)

sum_weights = weights_xgb+weights_lgbm+weights_cat

weights_xgb = weights_xgb/sum_weights
weights_lgbm = weights_lgbm/sum_weights
weights_cat = weights_cat/sum_weights

In [ ]:
# Predict results on training set.

comparison_df = full_df.loc[(full_df['year']==2017) & (full_df['date'] <= prediction_date),:]

for model_type in ('xgb', 'lgbm', 'cat'):
    if model_type == 'xgb':
        models = models_xgb
    elif model_type == 'lgbm':
        models = models_lgbm
    elif model_type == 'cat':
        models = models_cat
    for i in models.keys():
        print(i)
        y = models[i].predict(comparison_df[feature_list[i-1]])
        y = np.exp(y)-1
        y = np.clip(y, a_min = 0, a_max=None)
        comparison_df[f'{model_type}_pred_sales_step_{i}'] = y

In [ ]:
comparison_df.to_csv('/kaggle/working/comparison_df.csv',index=True)

In [ ]:
# Average predictions weighted by the performance-based weights

for i in range(1,17):
    comparison_df[f'avg_pred_sales_step_{i}'] = comparison_df[[f'xgb_pred_sales_step_{i}',
                                                              f'lgbm_pred_sales_step_{i}',
                                                              f'cat_pred_sales_step_{i}']].mean(axis=1)
    comparison_df[f'w_avg_pred_sales_step_{i}'] = weights_xgb[i-1]*comparison_df[f'xgb_pred_sales_step_{i}']+ weights_lgbm[i-1]*comparison_df[f'lgbm_pred_sales_step_{i}']+ weights_cat[i-1]*comparison_df[f'cat_pred_sales_step_{i}']

In [ ]:
# Examples

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

comparison_df.head()
comparison_df[comparison_df['date']=='2017-08-22'].head()

The following cell sets identifies stores and families that didn't sell any product during 2017, and then sets the predictions for these time-series to 0. The prediction should give negligible predictios anyway, but the real values are most likely 0 so we use that.

In [ ]:
# Predict 0 if there were no sales in 2017 for a store_nbr, family

no_sales_2017_df = comparison_df.groupby(['store_nbr', 'family'])[['sales']].sum()
no_sales_2017_df = no_sales_2017_df.reset_index()
no_sales_2017_df = no_sales_2017_df.rename(columns={'sales':'sum_2017_sales'})
no_sales_2017_df = no_sales_2017_df[no_sales_2017_df['sum_2017_sales']==0]

comparison_df = pd.merge(left=comparison_df,
                         right=no_sales_2017_df[['store_nbr','family','sum_2017_sales']],
                         on=['store_nbr','family'],
                         how='left')

for pred_type in ('avg', 'w_avg', 'xgb', 'lgbm', 'cat'):
    for i in range(1,17):
        comparison_df[f'{pred_type}_pred_sales_step_{i}'] = comparison_df['sum_2017_sales'].fillna(comparison_df[f'{pred_type}_pred_sales_step_{i}'])

In [ ]:
# Transpose predictions and join to test_df

prediction_df = comparison_df[comparison_df['date']== prediction_date]
prediction_df = prediction_df[['store_nbr', 'family'] 
                                +[f'avg_pred_sales_step_{i}' for i in range(1,17)]
                                +[f'w_avg_pred_sales_step_{i}' for i in range(1,17)]
                                +[f'xgb_pred_sales_step_{i}' for i in range(1,17)]
                                +[f'lgbm_pred_sales_step_{i}' for i in range(1,17)]
                                +[f'cat_pred_sales_step_{i}' for i in range(1,17)]]

# Merging predicted sales data from prediction_date
result_df = test_df.reset_index()
result_df = pd.merge(left=result_df,
                     right=prediction_df,
                     on=['store_nbr', 'family'],how='left')

# Choosing i-th prediction on day i + prediction_date
result_df['difference'] = (result_df['date']-pd.to_datetime(prediction_date)).dt.days

for pred_type in ('avg', 'w_avg', 'xgb', 'lgbm', 'cat'):
    col_names = result_df['difference'].astype(str).radd(f'{pred_type}_pred_sales_step_')
    col_codes, unique_cols = pd.factorize(col_names)
    matrix = result_df[unique_cols].to_numpy()
    result_df[f'{pred_type}_pred_sales'] = matrix[np.arange(len(result_df)), col_codes]

**Examples**

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

result_df.tail()

In [ ]:
str_nbr = 10
fml = 'BEVERAGES'

train_condition = (full_df['store_nbr']==str_nbr) & (full_df['family']==fml)
test_condition = (result_df['store_nbr']==str_nbr) & (result_df['family']==fml)

plt.figure()
axs = full_df.loc[train_condition,:].plot(x='date', y='sales_step_1',xlim = ('2017-07-01','2017-08-31')) #, ylim=(0,20)
axs = result_df.loc[test_condition,:].plot(x='date', y='avg_pred_sales_step_1', ax=axs)
#axs = full_df.loc[train_condition,:].plot(x='date', y='MA_last_month', ax=axs) #
#axs = result_df.loc[test_condition,:].plot(x='date', y='predicted_sales', ax=axs) #
plt.show()

**Evaluation**

Here, we compute the F-score of different features to help understand the contribution of each feature to the performance. It helps us understand if any features are redundant, which helps simplify the model and avoid overfitting. It also reinforces our trust in the model if we believe that the order of feature importance makes sense.

In [ ]:
from xgboost import plot_importance

for i in range(1,17):
    model = models_xgb[i]
    plt.figure()
    plot_importance(model, title=f"Target {i} Feature Importance", max_num_features=30) # 
    plt.show()

**Submission**

The rest of the code is just the organization of the prediction in a format suitable for submission.

In [ ]:
result_df = result_df[['id','date','store_nbr','family','avg_pred_sales','w_avg_pred_sales','xgb_pred_sales','lgbm_pred_sales','cat_pred_sales']]
result_df.to_csv('/kaggle/working/result_df.csv',index=True)
result_df.head()

In [ ]:
# Check submission file

chosen_pred = 'cat_pred_sales' # options: 'avg_pred_sales', 'w_avg_pred_sales','xgb_pred_sales','lgbm_pred_sales','cat_pred_sales'

to_submit = result_df[['id', chosen_pred]].rename(columns={chosen_pred:'sales'})
to_submit.head()

In [ ]:
# Submission

to_submit.to_csv('/kaggle/working/submission.csv',index=False)

**Submission using predictions-df**

In [ ]:
# # Run first 5 cells. Then:

# predictions_df = pd.read_csv('/kaggle/input/predictions-df/comparison_df (2).csv')

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 150)

# predictions_df.head()

In [ ]:
# # No weights available, so excluded.

# for i in range(1,17):
#     predictions_df[f'avg_pred_sales_step_{i}'] = predictions_df[[f'xgb_pred_sales_step_{i}',
#                                                               f'lgbm_pred_sales_step_{i}',
#                                                               f'cat_pred_sales_step_{i}']].mean(axis=1)

# # Transpose predictions and join to test_df

# # Predict 0 if there were no sales in 2017 for a store_nbr, family

# comparison_df = full_df.loc[full_df['year']==2017,:]
# no_sales_2017_df = comparison_df.groupby(['store_nbr', 'family'])[['sales']].sum()
# no_sales_2017_df = no_sales_2017_df.reset_index()
# no_sales_2017_df = no_sales_2017_df.rename(columns={'sales':'sum_2017_sales'})
# no_sales_2017_df = no_sales_2017_df[no_sales_2017_df['sum_2017_sales']==0]

# predictions_df = pd.merge(left=predictions_df,
#                          right=no_sales_2017_df[['store_nbr','family','sum_2017_sales']],
#                          on=['store_nbr','family'],
#                          how='left')

# for pred_type in ('avg', 'xgb', 'lgbm', 'cat'):
#     for i in range(1,17):
#         predictions_df[f'{pred_type}_pred_sales_step_{i}'] = predictions_df['sum_2017_sales'].fillna(predictions_df[f'{pred_type}_pred_sales_step_{i}'])

# #prediction_df = comparison_df[comparison_df['date']== prediction_date]
# predictions_df = predictions_df[['store_nbr', 'family'] 
#                                 +[f'avg_pred_sales_step_{i}' for i in range(1,17)]
#                                 +[f'xgb_pred_sales_step_{i}' for i in range(1,17)]
#                                 +[f'lgbm_pred_sales_step_{i}' for i in range(1,17)]
#                                 +[f'cat_pred_sales_step_{i}' for i in range(1,17)]]

# # Merging predicted sales data from prediction_date
# result_df = test_df.reset_index()
# result_df = pd.merge(left=result_df,
#                      right=predictions_df,
#                      on=['store_nbr', 'family'],how='left')

# # Choosing i-th prediction on day i + prediction_date
# result_df['difference'] = (result_df['date']-pd.to_datetime(prediction_date)).dt.days

# for pred_type in ('avg', 'xgb', 'lgbm', 'cat'):
#     col_names = result_df['difference'].astype(str).radd(f'{pred_type}_pred_sales_step_')
#     col_codes, unique_cols = pd.factorize(col_names)
#     matrix = result_df[unique_cols].to_numpy()
#     result_df[f'{pred_type}_pred_sales'] = matrix[np.arange(len(result_df)), col_codes]

# result_df.head()

In [ ]:
# # Check submission file

# chosen_pred = 'lgbm_pred_sales' # options: 'avg_pred_sales', 'w_avg_pred_sales','xgb_pred_sales','lgbm_pred_sales','cat_pred_sales'

# to_submit = result_df[['id', chosen_pred]].rename(columns={chosen_pred:'sales'})
# to_submit.head()

In [ ]:
# # Submission

# to_submit.to_csv('/kaggle/working/submission.csv',index=False)